# Qualité de l'air mesurée dans la station Châtelet

Source de données utilisée (fichiers CSV en open data): https://data.iledefrance.fr/explore/dataset/qualite-de-lair-mesuree-dans-la-station-chatelet/

**TODO**: l'index du *dataframe* devrait être constitué de la fusion des deux premières colonnes avec le bon *dtype* (un *datetime* Pandas).

## Récupération des données

In [ ]:
%matplotlib inline
#%matplotlib notebook

import matplotlib
matplotlib.rcParams['figure.figsize'] = (9, 9)

import pandas as pd

In [ ]:
def conv_func(s):
    s = s.replace('<', '')
    if s == 'ND':
        return np.nan
    elif s.strip() == '':
        return np.nan
    else:
        return float(s)

In [ ]:
url = "https://data.iledefrance.fr/explore/dataset/qualite-de-lair-mesuree-dans-la-station-chatelet/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true"

#dtype_dict = {'NO': np.float64,
#              'NO2': np.float64,
#              'PM10': np.float64,
#              'CO2': np.float64,
#              'TEMP': np.float64,
#              'HUMI': np.float64}

converter_dict = {'NO': conv_func,
                  'NO2': conv_func,
                  'PM10': conv_func,
                  'CO2': conv_func,
                  'TEMP': conv_func,
                  'HUMI': conv_func}

df = pd.read_csv(url,
                 #encoding='iso-8859-1',
                 index_col=0,
                 sep=';',
                 decimal=',',
                 parse_dates=["DATE/HEURE"],
                 #dtype=dtype_dict,
                 #na_values='ND',
                 converters=converter_dict)

#df.astype(dtype_dict)
df

In [ ]:
df = df.sort_index()

In [ ]:
df.index

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.loc[:,['PM10', 'CO2']].plot(figsize=(16,6))

In [ ]:
df.PM10.plot(figsize=(18,6))

In [ ]:
df.PM10.resample('1D').mean().plot(figsize=(18,6))

In [ ]:
df.PM10.rolling('7D').mean().plot(figsize=(18,6))

In [ ]:
df.PM10.resample('1M').mean().plot(figsize=(18,6))

In [ ]:
ax = df.PM10.groupby(df.index.time).mean().plot(figsize=(18,6), color="blue")

ax.set_xlabel("Time")

ax2 = ax.twinx()
df.TEMP.groupby(df.index.time).mean().plot(ax=ax2, color="red")

ax.legend(loc='upper left')
ax2.legend(loc='upper right');

In [ ]:
ax = df.PM10.groupby(df.index.weekday).mean().plot(figsize=(18,6), color="blue")

ax.set_xlabel("Weekday")

ax2 = ax.twinx()
df.TEMP.groupby(df.index.weekday).mean().plot(ax=ax2, color="red")

ax.legend(loc='upper left')
ax2.legend(loc='upper right');

In [ ]:
ax = df.PM10.groupby(df.index.month).mean().plot(figsize=(18,6), color="blue")

ax.set_xlabel("Month")

ax2 = ax.twinx()
df.TEMP.groupby(df.index.month).mean().plot(ax=ax2, color="red")

ax.legend(loc='upper left')
ax2.legend(loc='upper right');

In [ ]:
df.loc[:,['NO', 'NO2']].plot(figsize=(16,6))

In [ ]:
df.loc[:,['TEMP', 'HUMI']].plot(figsize=(16,6))